# Fetch historical candlestick data

## Dependencies initialization

In [1]:
from datetime import datetime, timezone
import os

from dotenv import load_dotenv
from gate_api import Configuration, ApiClient, FuturesApi
from gate_api.models.futures_candlestick import FuturesCandlestick
import pandas as pd

# Load environment variables from .env file
load_dotenv()

GATE_URL_HOST_LIVE = os.getenv("GATE_URL_HOST_LIVE")

SYMBOL_LIST = os.getenv("SYMBOL_LIST").split(",")

# Dependencies initialization
gate_configuration = Configuration(
    host=GATE_URL_HOST_LIVE,
)
gate_client = ApiClient(configuration=gate_configuration)
gate_futures_api = FuturesApi(api_client=gate_client)

## Methods to fetch and save historical data

In [2]:
def fetch_futures_candlesticks(
    symbol: str, timeframe: str, start_time: datetime, end_time: datetime
) -> list[FuturesCandlestick]:
    """
    Fetches up to 2000 candlesticks per request and concatenates results until the end_time is reached.
    On each loop, updates end_time to the timestamp of the first returned candle minus one interval.
    """
    result = []
    end_time = int(end_time.timestamp())
    interval_map = {
        "1s": 1,
        "5s": 5,
        "15s": 15,
        "30s": 30,
        "1m": 60,
        "5m": 300,
        "15m": 900,
        "30m": 1800,
        "1h": 3600,
        "4h": 14400,
        "8h": 28800,
        "1d": 86400,
    }
    interval_seconds = interval_map.get(timeframe)

    while True:
        candles = gate_futures_api.list_futures_candlesticks(
            settle="usdt",
            contract=symbol,
            interval=timeframe,
            _from=int(start_time.timestamp()),
            to=end_time,
        )
        if not candles:
            break
        result = candles + result
        # If less than 2000 returned, we're done
        if len(candles) < 2000:
            break
        # Update end_time to the first candle's timestamp minus one interval
        first_ts = int(candles[0].t)
        end_time = first_ts - interval_seconds
    return result


def candlesticks_to_dataframe(candle_list: list[FuturesCandlestick]) -> pd.DataFrame:
    data = [
        {
            "t": int(candle.t),
            "o": candle.o,
            "h": candle.h,
            "l": candle.l,
            "c": candle.c,
            "v": candle.v,
        }
        for candle in candle_list
    ]
    return pd.DataFrame(data)

def save_candlesticks_to_csv(
    candle_dataframe: pd.DataFrame, symbol: str, timeframe: str
):
    filename = f"{symbol}_{timeframe}_candlesticks.csv"
    candle_dataframe.to_csv(filename, index=False)
    print(f"Saved {len(candle_dataframe)} candles to {filename}")

## Download historical candlestick data

This part would be used to calculation of ATR and Renko bricks.

In [3]:
for symbol in SYMBOL_LIST:
    candle_list = fetch_futures_candlesticks(
        symbol=symbol,
        timeframe="1h",
        start_time=datetime(2024, 10, 1, 0, 0, 0, tzinfo=timezone.utc),
        end_time=datetime(2025, 1, 1, 0, 0, 0, tzinfo=timezone.utc)
    )
    candle_df = candlesticks_to_dataframe(candle_list)
    save_candlesticks_to_csv(candle_df, symbol, "1h")

Saved 2209 candles to BTC_USDT_1h_candlesticks.csv
Saved 2209 candles to ETH_USDT_1h_candlesticks.csv
Saved 2209 candles to XRP_USDT_1h_candlesticks.csv
Saved 2209 candles to SOL_USDT_1h_candlesticks.csv
Saved 2209 candles to DOGE_USDT_1h_candlesticks.csv


This part would be used to simulate the price stream for the backtest.

In [9]:
for symbol in SYMBOL_LIST:
    candle_list = fetch_futures_candlesticks(
        symbol=symbol,
        timeframe="1m",
        start_time=datetime(2025, 1, 1, 0, 1, 0, tzinfo=timezone.utc),
        end_time=datetime(2025, 7, 1, 0, 0, 0, tzinfo=timezone.utc)
    )
    candle_df = candlesticks_to_dataframe(candle_list)
    save_candlesticks_to_csv(candle_df, symbol, "1m")

Saved 260640 candles to BTC_USDT_1m_candlesticks.csv
Saved 260640 candles to ETH_USDT_1m_candlesticks.csv
Saved 260640 candles to XRP_USDT_1m_candlesticks.csv
Saved 260640 candles to SOL_USDT_1m_candlesticks.csv
Saved 260640 candles to DOGE_USDT_1m_candlesticks.csv
